In [39]:
import pandas as pd
from h3 import h3
import folium

In [74]:
def visualize_hexagons(hexagons, color='#bb111166', folium_map=None):
    polylines = {}
    lat = []
    lng= []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0] #список линий, которые формируют гекс
        lat.extend(map(lambda v:v[0], polyline)) #все широты в один список, для нахождения средней точки
        lng.extend(map(lambda v:v[1], polyline)) 
        polylines[hex] = polyline

    if folium_map is None:
            m = folium.Map(locations=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=8, tiles='cartodbpositron')
    else:    
            m = folium_map 
    for polyline in polylines.items():
            my_Polyline = folium.PolyLine(locations = polyline[1],
                                          weight = 3,
                                          color = color,
                                          fill = True,
                                          fill_color = "red",                                          
                                          popup = 'hex: ' + polyline[0],
                                          tooltip = polyline[0])
            m.add_child(my_Polyline)
    return m   

In [43]:
census_h3 = pd.read_excel('trans_hexes.xlsx')

In [65]:
hexes_9 = list(set(census_h3['h3_id']))
hexes_8 = list(set(census_h3['h3_id_8']))
hexes_7 = list(set(census_h3['h3_id_7']))
hexes_6 = list(set(census_h3['h3_id_6']))
hexes_5 = list(set(census_h3['h3_id_5']))
hexes_4 = list(set(census_h3['h3_id_4']))
hexes_3 = list(set(census_h3['h3_id_3']))

In [47]:
hexes_3

['8310c3fffffffff',
 '8310d5fffffffff',
 '8310eafffffffff',
 '8310dcfffffffff',
 '8310cafffffffff',
 '8310e1fffffffff',
 '8310c1fffffffff',
 '8310c5fffffffff',
 '8310ddfffffffff',
 '8310c0fffffffff',
 '8310cefffffffff',
 '8310e8fffffffff',
 '8310c8fffffffff',
 '8310c6fffffffff',
 '8310c2fffffffff',
 '8310d9fffffffff',
 '8310eefffffffff']

In [78]:
h = ['8910dc93603ffff','8910dc9360fffff','8910dc9361bffff']
#h = ['8910dc9249bffff']
m = visualize_hexagons(h)
display(m)